## Business Case: Delhivery - Feature Engineering 
The Data team builds intelligence and capabilities using this data that helps them to widen the gap between the quality, efficiency, and profitability of their business versus their competitors.
## Problem Statement definition: 
The company wants to understand and process the data coming out of data engineering pipelines:
• Clean, sanitize and manipulate data to get useful features out of raw fields
• Make sense out of the raw data and help the data science team to build forecasting models on it

* data - tells whether the data is testing or training data
* trip_creation_time – Timestamp of trip creation
* route_schedule_uuid – Unique Id for a particular route schedule
* route_type – Transportation type
    * FTL – Full Truck Load: FTL shipments get to the destination sooner, as the truck is making no other pickups or drop-offs along the way
    * Carting: Handling system consisting of small vehicles (carts)
* trip_uuid - Unique ID given to a particular trip (A trip may include different source and destination centers)
* source_center - Source ID of trip origin
* source_name - Source Name of trip origin
* destination_cente – Destination ID
* destination_name – Destination Name
* od_start_time – Trip start time
* od_end_time – Trip end time
* start_scan_to_end_scan – Time taken to deliver from source to destination
* is_cutoff – Unknown field
* cutoff_factor – Unknown field
* cutoff_timestamp – Unknown field
* actual_distance_to_destination – Distance in Kms between source and destination warehouse
* actual_time – Actual time taken to complete the delivery (Cumulative)
* osrm_time – An open-source routing engine time calculator which computes the shortest path between points in a given map (Includes usual traffic, distance through major and minor roads) and gives the time (Cumulative)
* osrm_distance – An open-source routing engine which computes the shortest path between points in a given map (Includes usual traffic, distance through major and minor roads) (Cumulative)
* factor – Unknown field
* segment_actual_time – This is a segment time. Time taken by the subset of the package delivery
* segment_osrm_time – This is the OSRM segment time. Time taken by the subset of the package delivery
* segment_osrm_distance – This is the OSRM distance. Distance covered by subset of the package delivery
* segment_factor – Unknown field

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
from  scipy.stats import ttest_ind,f_oneway,chi2_contingency
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import levene

In [4]:
delhivery_df = pd.read_csv("delhivery_data.csv")

print('Shape of the data set is as follows: ')
print('No. of Rows: '+ str(delhivery_df.shape[0]))
print('No. of Columns: '+ str(delhivery_df.shape[1]))
print('-------------------------------------------')

Shape of the data set is as follows: 
No. of Rows: 144867
No. of Columns: 24
-------------------------------------------


In [5]:
delhivery_df.describe().T  

count        mean          std         min  \
start_scan_to_end_scan          144867.0  961.262986  1037.012769   20.000000   
cutoff_factor                   144867.0  232.926567   344.755577    9.000000   
actual_distance_to_destination  144867.0  234.073372   344.990009    9.000045   
actual_time                     144867.0  416.927527   598.103621    9.000000   
osrm_time                       144867.0  213.868272   308.011085    6.000000   
osrm_distance                   144867.0  284.771297   421.119294    9.008200   
factor                          144867.0    2.120107     1.715421    0.144000   
segment_actual_time             144867.0   36.196111    53.571158 -244.000000   
segment_osrm_time               144867.0   18.507548    14.775960    0.000000   
segment_osrm_distance           144867.0   22.829020    17.860660    0.000000   
segment_factor                  144867.0    2.218368     4.847530  -23.444444   

                                       25%         50%          75%  \
start_scan_to_end_scan          161.000000  449.000000  1634.000000   
cutoff_factor                    22.000000   66.000000   286.000000   
actual_distance_to_destination   23.355874   66.126571   286.708875   
actual_time                      51.000000  132.000000   513.000000   
osrm_time                        27.000000   64.000000   257.000000   
osrm_distance                    29.914700   78.525800   343.193250   
factor                            1.604264    1.857143     2.213483   
segment_actual_time              20.000000   29.000000    40.000000   
segment_osrm_time                11.000000   17.000000    22.000000   
segment_osrm_distance            12.070100   23.513000    27.813250   
segment_factor                    1.347826    1.684211     2.250000   

                                        max  
start_scan_to_end_scan          7898.000000  
cutoff_factor                   1927.000000  
actual_distance_to_destination  1927.447705  
actual_time                     4532.000000  
osrm_time                       1686.000000  
osrm_distance                   2326.199100  
factor                            77.387097  
segment_actual_time             3051.000000  
segment_osrm_time               1611.000000  
segment_osrm_distance           2191.403700  
segment_factor                   574.250000

In [6]:
print("Number of unique values for each column ")
print("------------------------------------------")
for column in list(delhivery_df.columns):
    print(column+ " :  "+ str(delhivery_df[''+column+''].value_counts().index.nunique()) )

Number of unique values for each column 
------------------------------------------
data :  2
trip_creation_time :  14817
route_schedule_uuid :  1504
route_type :  2
trip_uuid :  14817
source_center :  1508
source_name :  1498
destination_center :  1481
destination_name :  1468
od_start_time :  26369
od_end_time :  26369
start_scan_to_end_scan :  1915
is_cutoff :  2
cutoff_factor :  501
cutoff_timestamp :  93180
actual_distance_to_destination :  144515
actual_time :  3182
osrm_time :  1531
osrm_distance :  138046
factor :  45641
segment_actual_time :  747
segment_osrm_time :  214
segment_osrm_distance :  113799
segment_factor :  5675


In [7]:
#Values of attributes having 5 or less categories based on the above unique value counts

print("data unique values : ")
print(delhivery_df['data'].value_counts().index.to_list())
print("-------------------------")

print("route_type unique values ")
print(delhivery_df['route_type'].value_counts().index.to_list())
print("-------------------------")


print("Cutoff unique values ")
print(delhivery_df['is_cutoff'].value_counts().index.to_list())
print("-------------------------")



data unique values : 
['training', 'test']
-------------------------
route_type unique values 
['FTL', 'Carting']
-------------------------
Cutoff unique values 
[True, False]
-------------------------


In [8]:
delhivery_df.isna().sum()

data                                0
trip_creation_time                  0
route_schedule_uuid                 0
route_type                          0
trip_uuid                           0
source_center                       0
source_name                       293
destination_center                  0
destination_name                  261
od_start_time                       0
od_end_time                         0
start_scan_to_end_scan              0
is_cutoff                           0
cutoff_factor                       0
cutoff_timestamp                    0
actual_distance_to_destination      0
actual_time                         0
osrm_time                           0
osrm_distance                       0
factor                              0
segment_actual_time                 0
segment_osrm_time                   0
segment_osrm_distance               0
segment_factor                      0
dtype: int64

### Converting date time columns into datetime64

In [9]:

datetime_columns = ['trip_creation_time', 'od_start_time', 'od_end_time']
for i in datetime_columns:
    delhivery_df[i] = pd.to_datetime(delhivery_df[i])

#delhivery_df.info()
null_value_count = delhivery_df.isna().sum().to_dict()
print("Columns having null values: ")
print("----------------------------")
for k in null_value_count.keys():
    if null_value_count[k] != 0:
        print(k)

Columns having null values: 
----------------------------
source_name
destination_name


## As we see "source_name" and "destination_name" have null values
* We need to Replace source names with a placeholder location value for each of the source_centers having null source_names
* For destination name, 
    *  Find the placeholder we already assigned to source_center having null source_name and assign the same to destination_name as well (to prevent    duplicate destination_names for the same destination_center )
    * Replace destination names with a placeholder location value for each of the destination_centers having null destination_name

### Source name population with placeholders

In [10]:
import numpy as np
null_source_names = delhivery_df[delhivery_df["source_name"].isna() == True]["source_center"].unique().tolist()

print("Source center having null source names: ")
print(null_source_names)
num = 0 
for sc in null_source_names:
    #print(sc)
    delhivery_df.loc[delhivery_df["source_center"]== sc, "source_name" ] = f"location_{num}"
    num+=1

print("-------------------")
print("Replacing source names with a placeholder location value for each of the source_centers having null source_names")
for i in null_source_names:
    sc_name = delhivery_df[(delhivery_df["source_center"]== i) ]["source_name"].unique()
    print(f"Source_name: {sc_name} and source_center : {i}")
print(f"location_num_max : {num}")
location_num_max = num
# ["source_name","source_center"]

Source center having null source names: 
['IND342902A1B', 'IND577116AAA', 'IND282002AAD', 'IND465333A1B', 'IND841301AAC', 'IND509103AAC', 'IND126116AAA', 'IND331022A1B', 'IND505326AAB', 'IND852118A1B']
-------------------
Replacing source names with a placeholder location value for each of the source_centers having null source_names
Source_name: ['location_0'] and source_center : IND342902A1B
Source_name: ['location_1'] and source_center : IND577116AAA
Source_name: ['location_2'] and source_center : IND282002AAD
Source_name: ['location_3'] and source_center : IND465333A1B
Source_name: ['location_4'] and source_center : IND841301AAC
Source_name: ['location_5'] and source_center : IND509103AAC
Source_name: ['location_6'] and source_center : IND126116AAA
Source_name: ['location_7'] and source_center : IND331022A1B
Source_name: ['location_8'] and source_center : IND505326AAB
Source_name: ['location_9'] and source_center : IND852118A1B
location_num_max : 10


### Destination name population and lookup to avoid duplicate population of destination names from source names  

In [11]:
import numpy as np
null_destination_names = delhivery_df[delhivery_df["destination_name"].isna() == True]["destination_center"].unique().tolist()


print("Destination having null destination names: ")
print(null_destination_names)
num = 0 
for dc in null_destination_names:
    #print(sc))
    if dc in delhivery_df[delhivery_df["source_center"]== dc]["source_center"].unique().tolist():
        sc_name = delhivery_df[(delhivery_df["source_center"]== dc) ]["source_name"].unique()
        #print(sc_name)
        delhivery_df.loc[delhivery_df["destination_center"]== dc, "destination_name" ] = sc_name[0]
        print(f"Destination_name: {sc_name[0]} and destination_center : {dc}")
        


null_destination_names_after_source_name_reference = delhivery_df[delhivery_df["destination_name"].isna() == True]["destination_center"].unique().tolist()

print("Destination center having null destination names: ")
print(null_destination_names_after_source_name_reference)
num = 0 
for dc in null_destination_names_after_source_name_reference:
    #print(sc)
    delhivery_df.loc[delhivery_df["destination_center"]== dc, "destination_name" ] = f"location_{location_num_max}"
    location_num_max+=1

print("-------------------")
print("Replacing destination names with a placeholder location value for each of the destination_centers having null destination_names and does not have a prior source name assigned for the same source_center")
for i in null_destination_names_after_source_name_reference:
    dc_name = delhivery_df[(delhivery_df["destination_center"]== i) ]["destination_name"].unique()
    print(f"destination_name: {dc_name} and destination_center : {i}")



Destination having null destination names: 
['IND342902A1B', 'IND577116AAA', 'IND282002AAD', 'IND465333A1B', 'IND841301AAC', 'IND505326AAB', 'IND852118A1B', 'IND126116AAA', 'IND509103AAC', 'IND221005A1A', 'IND250002AAC', 'IND331001A1C', 'IND122015AAC']
Destination_name: location_0 and destination_center : IND342902A1B
Destination_name: location_1 and destination_center : IND577116AAA
Destination_name: location_2 and destination_center : IND282002AAD
Destination_name: location_3 and destination_center : IND465333A1B
Destination_name: location_4 and destination_center : IND841301AAC
Destination_name: location_8 and destination_center : IND505326AAB
Destination_name: location_9 and destination_center : IND852118A1B
Destination_name: location_6 and destination_center : IND126116AAA
Destination_name: location_5 and destination_center : IND509103AAC
Destination center having null destination names: 
['IND221005A1A', 'IND250002AAC', 'IND331001A1C', 'IND122015AAC']
-------------------
Replacin

In [12]:
delhivery_df.isna().sum()

data                              0
trip_creation_time                0
route_schedule_uuid               0
route_type                        0
trip_uuid                         0
source_center                     0
source_name                       0
destination_center                0
destination_name                  0
od_start_time                     0
od_end_time                       0
start_scan_to_end_scan            0
is_cutoff                         0
cutoff_factor                     0
cutoff_timestamp                  0
actual_distance_to_destination    0
actual_time                       0
osrm_time                         0
osrm_distance                     0
factor                            0
segment_actual_time               0
segment_osrm_time                 0
segment_osrm_distance             0
segment_factor                    0
dtype: int64

#### Handling of null values done

## Merging and aggregation of necessary fields

We can define each segment as same [trip_id, source_center, destination_center]


Segment related columns as:
* segment_actual_time
* segment_osrm_time
* segment_osrm_distance



Let's create a new id to group them together using a single columns , that indicates:
* trip_id + source_center + destination_center

In [67]:
# Segment defined as trips having same trip_id, source and destination
delhivery_df ["trip_segment_id"]= delhivery_df["trip_uuid"] + delhivery_df["source_center"] +  delhivery_df["destination_center"] 

# We take cum sum for each of the segment and then use the last value to get the total time required to complete that segment
delhivery_df["segment_actual_time_cum_sum"]= delhivery_df.groupby("trip_segment_id")["segment_actual_time"].cumsum()
delhivery_df["segment_osrm_time_cum_sum"]= delhivery_df.groupby("trip_segment_id")["segment_osrm_time"].cumsum()
delhivery_df["segment_osrm_distance_cum_sum"]= delhivery_df.groupby("trip_segment_id")["segment_osrm_distance"].cumsum()

delhivery_df[["segment_actual_time_cum_sum","segment_osrm_time_cum_sum","segment_osrm_distance_cum_sum"]]


segment_actual_time_cum_sum  segment_osrm_time_cum_sum  \
0                              14.0                       11.0   
1                              24.0                       20.0   
2                              40.0                       27.0   
3                              61.0                       39.0   
4                              67.0                       44.0   
...                             ...                        ...   
144862                         92.0                       94.0   
144863                        118.0                      115.0   
144864                        138.0                      149.0   
144865                        155.0                      176.0   
144866                        423.0                      185.0   

        segment_osrm_distance_cum_sum  
0                             11.9653  
1                             21.7243  
2                             32.5395  
3                             45.5619  
4                             49.4772  
...                               ...  
144862                        65.3487  
144863                        82.7212  
144864                       103.4265  
144865                       122.3150  
144866                       131.1238  

[144867 rows x 3 columns]

In [68]:
dh_df_g1 = delhivery_df.groupby(by= ["trip_segment_id"], as_index=False).agg({
    "data": "first",
    'route_type' : 'first',
    "trip_creation_time": "first",
    "route_schedule_uuid": "first",
    "trip_uuid": "first",

    # We want to preserve the first source name
    "source_name": "first",
    "source_center": "first",
    
    "destination_name":"first",
    "destination_center":"first",
    
    "od_start_time": "first",
    "od_end_time": "first",
    "start_scan_to_end_scan": "first",

    "actual_distance_to_destination": "last",
    "actual_time": "last",

    "osrm_time": "last",
    "osrm_distance": "last",

    # Since we have computed cum_sum with each of the newly created segment id
    "segment_actual_time_cum_sum": "last",
    "segment_osrm_time_cum_sum":"last",
    "segment_osrm_distance_cum_sum":"last"
})
dh_df_g1

trip_segment_id      data route_type  \
0      trip-153671041653548748IND209304AAAIND000000ACB  training        FTL   
1      trip-153671041653548748IND462022AAAIND209304AAA  training        FTL   
2      trip-153671042288605164IND561203AABIND562101AAA  training    Carting   
3      trip-153671042288605164IND572101AAAIND561203AAB  training    Carting   
4      trip-153671043369099517IND000000ACBIND160002AAC  training        FTL   
...                                                ...       ...        ...   
26363  trip-153861115439069069IND628204AAAIND627657AAA      test    Carting   
26364  trip-153861115439069069IND628613AAAIND627005AAA      test    Carting   
26365  trip-153861115439069069IND628801AAAIND628204AAA      test    Carting   
26366  trip-153861118270144424IND583119AAAIND583101AAA      test        FTL   
26367  trip-153861118270144424IND583201AAAIND583119AAA      test        FTL   

              trip_creation_time  \
0     2018-09-12 00:00:16.535741   
1     2018-09-12 00:00:16.535741   
2     2018-09-12 00:00:22.886430   
3     2018-09-12 00:00:22.886430   
4     2018-09-12 00:00:33.691250   
...                          ...   
26363 2018-10-03 23:59:14.390954   
26364 2018-10-03 23:59:14.390954   
26365 2018-10-03 23:59:14.390954   
26366 2018-10-03 23:59:42.701692   
26367 2018-10-03 23:59:42.701692   

                                     route_schedule_uuid  \
0      thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...   
1      thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...   
2      thanos::sroute:3a1b0ab2-bb0b-4c53-8c59-eb2a2c0...   
3      thanos::sroute:3a1b0ab2-bb0b-4c53-8c59-eb2a2c0...   
4      thanos::sroute:de5e208e-7641-45e6-8100-4d9fb1e...   
...                                                  ...   
26363  thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...   
26364  thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...   
26365  thanos::sroute:c5f2ba2c-8486-4940-8af6-d1d2a6a...   
26366  thanos::sroute:412fea14-6d1f-4222-8a5f-a517042...   
26367  thanos::sroute:412fea14-6d1f-4222-8a5f-a517042...   

                     trip_uuid                         source_name  \
0      trip-153671041653548748  Kanpur_Central_H_6 (Uttar Pradesh)   
1      trip-153671041653548748  Bhopal_Trnsport_H (Madhya Pradesh)   
2      trip-153671042288605164   Doddablpur_ChikaDPP_D (Karnataka)   
3      trip-153671042288605164       Tumkur_Veersagr_I (Karnataka)   
4      trip-153671043369099517       Gurgaon_Bilaspur_HB (Haryana)   
...                        ...                                 ...   
26363  trip-153861115439069069  Tirchchndr_Shnmgprm_D (Tamil Nadu)   
26364  trip-153861115439069069   Peikulam_SriVnktpm_D (Tamil Nadu)   
26365  trip-153861115439069069        Eral_Busstand_D (Tamil Nadu)   
26366  trip-153861118270144424       Sandur_WrdN1DPP_D (Karnataka)   
26367  trip-153861118270144424                  Hospet (Karnataka)   

      source_center                       destination_name destination_center  \
0      IND209304AAA          Gurgaon_Bilaspur_HB (Haryana)       IND000000ACB   
1      IND462022AAA     Kanpur_Central_H_6 (Uttar Pradesh)       IND209304AAA   
2      IND561203AAB      Chikblapur_ShntiSgr_D (Karnataka)       IND562101AAA   
3      IND572101AAA      Doddablpur_ChikaDPP_D (Karnataka)       IND561203AAB   
4      IND000000ACB         Chandigarh_Mehmdpur_H (Punjab)       IND160002AAC   
...             ...                                    ...                ...   
26363  IND628204AAA  Thisayanvilai_UdnkdiRD_D (Tamil Nadu)       IND627657AAA   
26364  IND628613AAA    Tirunelveli_VdkkuSrt_I (Tamil Nadu)       IND627005AAA   
26365  IND628801AAA     Tirchchndr_Shnmgprm_D (Tamil Nadu)       IND628204AAA   
26366  IND583119AAA                 Bellary_Dc (Karnataka)       IND583101AAA   
26367  IND583201AAA          Sandur_WrdN1DPP_D (Karnataka)       IND583119AAA   

                   od_start_time                od_end_time  \
0     2018-09-12 16:39:46.858469 2018-09-13 13:40:23.123744

In [ ]:
group_1 = ["trip_uuid","source_center","destination_center"]
dh_df_1 = delhivery_df.groupby(by= group_1, as_index=False).agg({
    "data": "first",
    'route_type' : 'first',
    "trip_creation_time": "first",
    "source_name": "first",
    "destination_name":"first",
    "od_start_time": "first",
    "od_end_time": "first",
    "start_scan_to_end_scan": "first",
    "actual_distance_to_destination": "last", # We want to know the actual distance of the last destination to get the whole distance
    "actual_time": "last",
    "osrm_time": "last",
    "osrm_distance": "last",
    "segment_actual_time": "sum",
    "segment_osrm_time":"sum",
    "segment_osrm_distance":"sum"
})
dh_df_1

dh_df_1[(dh_df_1["trip_uuid"] == "trip-153671041653548748") &	(dh_df_1["source_center"] == "IND209304AAA") & (dh_df_1["destination_center"] == "IND000000ACB")]


In [57]:
delhivery_df[(delhivery_df["trip_uuid"] == "trip-153671041653548748")]

data         trip_creation_time  \
124981  training 2018-09-12 00:00:16.535741   
124982  training 2018-09-12 00:00:16.535741   
124983  training 2018-09-12 00:00:16.535741   
124984  training 2018-09-12 00:00:16.535741   
124985  training 2018-09-12 00:00:16.535741   
124986  training 2018-09-12 00:00:16.535741   
124987  training 2018-09-12 00:00:16.535741   
124988  training 2018-09-12 00:00:16.535741   
124989  training 2018-09-12 00:00:16.535741   
124990  training 2018-09-12 00:00:16.535741   
124991  training 2018-09-12 00:00:16.535741   
124992  training 2018-09-12 00:00:16.535741   
124993  training 2018-09-12 00:00:16.535741   
124994  training 2018-09-12 00:00:16.535741   
124995  training 2018-09-12 00:00:16.535741   
124996  training 2018-09-12 00:00:16.535741   
124997  training 2018-09-12 00:00:16.535741   
124998  training 2018-09-12 00:00:16.535741   
124999  training 2018-09-12 00:00:16.535741   
125000  training 2018-09-12 00:00:16.535741   
125001  training 2018-09-12 00:00:16.535741   
125002  training 2018-09-12 00:00:16.535741   
125003  training 2018-09-12 00:00:16.535741   
125004  training 2018-09-12 00:00:16.535741   
125005  training 2018-09-12 00:00:16.535741   
125006  training 2018-09-12 00:00:16.535741   
125007  training 2018-09-12 00:00:16.535741   
125008  training 2018-09-12 00:00:16.535741   
125009  training 2018-09-12 00:00:16.535741   
125010  training 2018-09-12 00:00:16.535741   
125011  training 2018-09-12 00:00:16.535741   
125012  training 2018-09-12 00:00:16.535741   
125013  training 2018-09-12 00:00:16.535741   
125014  training 2018-09-12 00:00:16.535741   
125015  training 2018-09-12 00:00:16.535741   
125016  training 2018-09-12 00:00:16.535741   
125017  training 2018-09-12 00:00:16.535741   
125018  training 2018-09-12 00:00:16.535741   
125019  training 2018-09-12 00:00:16.535741   

                                      route_schedule_uuid route_type  \
124981  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124982  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124983  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124984  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124985  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124986  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124987  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124988  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124989  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124990  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124991  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124992  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124993  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124994  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124995  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124996  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124997  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124998  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
124999  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125000  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125001  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125002  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125003  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125004  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125005  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125006  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125007  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125008  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125009  thanos::sroute:d7c989ba-a29b-4a0b-b

In [58]:

delhivery_df[(delhivery_df["trip_uuid"] == "trip-153671041653548748") &	(delhivery_df["source_center"] == "IND209304AAA") & (delhivery_df["destination_center"] == "IND000000ACB")]

data         trip_creation_time  \
125002  training 2018-09-12 00:00:16.535741   
125003  training 2018-09-12 00:00:16.535741   
125004  training 2018-09-12 00:00:16.535741   
125005  training 2018-09-12 00:00:16.535741   
125006  training 2018-09-12 00:00:16.535741   
125007  training 2018-09-12 00:00:16.535741   
125008  training 2018-09-12 00:00:16.535741   
125009  training 2018-09-12 00:00:16.535741   
125010  training 2018-09-12 00:00:16.535741   
125011  training 2018-09-12 00:00:16.535741   
125012  training 2018-09-12 00:00:16.535741   
125013  training 2018-09-12 00:00:16.535741   
125014  training 2018-09-12 00:00:16.535741   
125015  training 2018-09-12 00:00:16.535741   
125016  training 2018-09-12 00:00:16.535741   
125017  training 2018-09-12 00:00:16.535741   
125018  training 2018-09-12 00:00:16.535741   
125019  training 2018-09-12 00:00:16.535741   

                                      route_schedule_uuid route_type  \
125002  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125003  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125004  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125005  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125006  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125007  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125008  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125009  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125010  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125011  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125012  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125013  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125014  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125015  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125016  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125017  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125018  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   
125019  thanos::sroute:d7c989ba-a29b-4a0b-b2f4-288cdc6...        FTL   

                      trip_uuid source_center  \
125002  trip-153671041653548748  IND209304AAA   
125003  trip-153671041653548748  IND209304AAA   
125004  trip-153671041653548748  IND209304AAA   
125005  trip-153671041653548748  IND209304AAA   
125006  trip-153671041653548748  IND209304AAA   
125007  trip-153671041653548748  IND209304AAA   
125008  trip-153671041653548748  IND209304AAA   
125009  trip-153671041653548748  IND209304AAA   
125010  trip-153671041653548748  IND209304AAA   
125011  trip-153671041653548748  IND209304AAA   
125012  trip-153671041653548748  IND209304AAA   
125013  trip-153671041653548748  IND209304AAA   
125014  trip-153671041653548748  IND209304AAA   
125015  trip-153671041653548748  IND209304AAA   
125016  trip-153671041653548748  IND209304AAA   
125017  trip-153671041653548748  IND209304AAA   
125018  trip-153671041653548748  IND209304AAA   
125019  trip-153671041653548748  IND209304AAA   

                               source_name destination_center  \
125002  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125003  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125004  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125005  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125006  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125007  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125008  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125009  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125010  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125011  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125012  Kanpur_Central_H_6 (Uttar Pradesh)       IND000000ACB   
125013  Kanpur_Central_H_6 (Uttar Pra

In [59]:
dh_df_1["trip_uuid"].value_counts()
dh_df_1[dh_df_1["trip_uuid"] == "trip-153758895506669465"]

trip_uuid source_center destination_center      data  \
13143  trip-153758895506669465  IND000000AAS       IND783370AAC  training   
13144  trip-153758895506669465  IND000000AET       IND000000AAS  training   
13145  trip-153758895506669465  IND781018AAB       IND781354AAA  training   
13146  trip-153758895506669465  IND781335AAA       IND000000AET  training   
13147  trip-153758895506669465  IND783301AAA       IND783348AAA  training   
13148  trip-153758895506669465  IND783348AAA       IND783129AAA  training   
13149  trip-153758895506669465  IND783370AAC       IND783301AAA  training   
13150  trip-153758895506669465  IND794101AAA       IND781018AAB  training   

      route_type         trip_creation_time                    source_name  \
13143        FTL 2018-09-22 04:02:35.066945  Bongaigaon_Chpaguri_D (Assam)   
13144        FTL 2018-09-22 04:02:35.066945     Barpeta_ShivBari_D (Assam)   
13145        FTL 2018-09-22 04:02:35.066945           Guwahati_Hub (Assam)   
13146        FTL 2018-09-22 04:02:35.066945     Nalbari_Bhgtpura_D (Assam)   
13147        FTL 2018-09-22 04:02:35.066945      Dhubri_Tetultol_D (Assam)   
13148        FTL 2018-09-22 04:02:35.066945   Bilasipara_Wardno5_D (Assam)   
13149        FTL 2018-09-22 04:02:35.066945   Kokrajhar_PigonDPP_D (Assam)   
13150        FTL 2018-09-22 04:02:35.066945    Tura_Sracmplx_D (Meghalaya)   

                    destination_name              od_start_time  \
13143   Kokrajhar_PigonDPP_D (Assam) 2018-09-22 16:35:26.103789   
13144  Bongaigaon_Chpaguri_D (Assam) 2018-09-22 14:01:19.587563   
13145      Rangia_HosptlRd_D (Assam) 2018-09-22 04:02:35.066945   
13146     Barpeta_ShivBari_D (Assam) 2018-09-22 12:03:37.925505   
13147   Bilasipara_Wardno5_D (Assam) 2018-09-22 20:15:28.551075   
13148    Lakhipur_WrdN4DPP_D (Assam) 2018-09-22 21:54:27.954685   
13149      Dhubri_Tetultol_D (Assam) 2018-09-22 17:57:08.164799   
13150           Guwahati_Hub (Assam) 2018-09-23 05:51:56.664783   

                     od_end_time  start_scan_to_end_scan  \
13143 2018-09-22 17:57:08.164799                    81.0   
13144 2018-09-22 16:35:26.103789                   154.0   
13145 2018-09-22 11:11:07.624473                   428.0   
13146 2018-09-22 14:01:19.587563                   117.0   
13147 2018-09-22 21:54:27.954685                    98.0   
13148 2018-09-23 00:38:12.844641                   163.0   
13149 2018-09-22 20:15:28.551075                   138.0   
13150 2018-09-23 12:40:42.519792                   408.0   

       actual_distance_to_destination  actual_time  osrm_time  osrm_distance  \
13143                       28.702820         77.0       30.0        42.6142   
13144                       48.550647        114.0       48.0        68.3918   
13145                       36.518932         80.0       54.0        64.7616   
13146                       46.115890         93.0       50.0        64.9010   
13147                       33.634861         60.0       32.0        43.2133   
13148                       24.171871        144.0       89.0       101.4926   
13149                       51.713578        116.0       47.0        66.9945   
13150                      161.625082        359.0      159.0       207.1968   

       segment_actual_time  segment_osrm_time  segment_osrm_distance  
13143                 76.0               30.0                42.6142  
13144                114.0               47.0                68.3918  
13145                 80.0               54.0                64.7616  
13146                 91.0               49.0                64.9010  
13147                 59.0               32.0                43.2133  
13148                143.0               97.0               124.3001  
13149                116.0               46.0                66.9945  
13150                355.0              162.0               213.1223

In [60]:
dh_df_1[dh_df_1["trip_uuid"] == "trip-153861115439069069"]


trip_uuid source_center destination_center  data  \
26361  trip-153861115439069069  IND627005AAA       IND628801AAA  test   
26362  trip-153861115439069069  IND627657AAA       IND628613AAA  test   
26363  trip-153861115439069069  IND628204AAA       IND627657AAA  test   
26364  trip-153861115439069069  IND628613AAA       IND627005AAA  test   
26365  trip-153861115439069069  IND628801AAA       IND628204AAA  test   

      route_type         trip_creation_time  \
26361    Carting 2018-10-03 23:59:14.390954   
26362    Carting 2018-10-03 23:59:14.390954   
26363    Carting 2018-10-03 23:59:14.390954   
26364    Carting 2018-10-03 23:59:14.390954   
26365    Carting 2018-10-03 23:59:14.390954   

                                 source_name  \
26361    Tirunelveli_VdkkuSrt_I (Tamil Nadu)   
26362  Thisayanvilai_UdnkdiRD_D (Tamil Nadu)   
26363     Tirchchndr_Shnmgprm_D (Tamil Nadu)   
26364      Peikulam_SriVnktpm_D (Tamil Nadu)   
26365           Eral_Busstand_D (Tamil Nadu)   

                            destination_name              od_start_time  \
26361           Eral_Busstand_D (Tamil Nadu) 2018-10-03 23:59:14.390954   
26362      Peikulam_SriVnktpm_D (Tamil Nadu) 2018-10-04 03:31:11.183797   
26363  Thisayanvilai_UdnkdiRD_D (Tamil Nadu) 2018-10-04 02:29:04.272194   
26364    Tirunelveli_VdkkuSrt_I (Tamil Nadu) 2018-10-04 04:16:39.894872   
26365     Tirchchndr_Shnmgprm_D (Tamil Nadu) 2018-10-04 01:44:53.808000   

                     od_end_time  start_scan_to_end_scan  \
26361 2018-10-04 01:44:53.808000                   105.0   
26362 2018-10-04 04:16:39.894872                    45.0   
26363 2018-10-04 03:31:11.183797                    62.0   
26364 2018-10-04 05:47:45.162682                    91.0   
26365 2018-10-04 02:29:04.272194                    44.0   

       actual_distance_to_destination  actual_time  osrm_time  osrm_distance  \
26361                       37.387664         60.0       45.0        46.2538   
26362                       17.373209         33.0       31.0        25.7087   
26363                       33.627182         51.0       41.0        42.5213   
26364                       33.673835         90.0       48.0        40.6080   
26365                       12.661945         30.0       14.0        16.0185   

       segment_actual_time  segment_osrm_time  segment_osrm_distance  
26361                 59.0               58.0                61.0753  
26362                 32.0               30.0                25.7087  
26363                 49.0               42.0                42.1431  
26364                 89.0               77.0                78.5869  
26365                 29.0               14.0                16.0184

In [61]:
df2 = dh_df_1.groupby(by = 'trip_uuid', as_index = False).agg({'source_center' : 'first',
                                                           'destination_center' : 'last',
                                                           'data' : 'first',
                                                           'route_type' : 'first',
                                                           'trip_creation_time' : 'first',
                                                           'source_name' : 'first',
                                                           'destination_name' : 'last',
                                                           'start_scan_to_end_scan' : 'sum',
                                                           'actual_distance_to_destination' : 'sum',
                                                           'actual_time' : 'sum',
                                                           'osrm_time' : 'sum',
                                                           'osrm_distance' : 'sum',
                                                           'segment_actual_time' : 'sum',
                                                           'segment_osrm_time' : 'sum',
                                                           'segment_osrm_distance' : 'sum'})
df2

trip_uuid source_center destination_center      data  \
0      trip-153671041653548748  IND209304AAA       IND209304AAA  training   
1      trip-153671042288605164  IND561203AAB       IND561203AAB  training   
2      trip-153671043369099517  IND000000ACB       IND000000ACB  training   
3      trip-153671046011330457  IND400072AAB       IND401104AAA  training   
4      trip-153671052974046625  IND583101AAA       IND583119AAA  training   
...                        ...           ...                ...       ...   
14812  trip-153861095625827784  IND160002AAC       IND160002AAC      test   
14813  trip-153861104386292051  IND121004AAB       IND121004AAA      test   
14814  trip-153861106442901555  IND208006AAA       IND208006AAA      test   
14815  trip-153861115439069069  IND627005AAA       IND628204AAA      test   
14816  trip-153861118270144424  IND583119AAA       IND583119AAA      test   

      route_type         trip_creation_time  \
0            FTL 2018-09-12 00:00:16.535741   
1        Carting 2018-09-12 00:00:22.886430   
2            FTL 2018-09-12 00:00:33.691250   
3        Carting 2018-09-12 00:01:00.113710   
4            FTL 2018-09-12 00:02:09.740725   
...          ...                        ...   
14812    Carting 2018-10-03 23:55:56.258533   
14813    Carting 2018-10-03 23:57:23.863155   
14814    Carting 2018-10-03 23:57:44.429324   
14815    Carting 2018-10-03 23:59:14.390954   
14816        FTL 2018-10-03 23:59:42.701692   

                               source_name  \
0       Kanpur_Central_H_6 (Uttar Pradesh)   
1        Doddablpur_ChikaDPP_D (Karnataka)   
2            Gurgaon_Bilaspur_HB (Haryana)   
3                 Mumbai Hub (Maharashtra)   
4                   Bellary_Dc (Karnataka)   
...                                    ...   
14812       Chandigarh_Mehmdpur_H (Punjab)   
14813         FBD_Balabhgarh_DPC (Haryana)   
14814   Kanpur_GovndNgr_DC (Uttar Pradesh)   
14815  Tirunelveli_VdkkuSrt_I (Tamil Nadu)   
14816        Sandur_WrdN1DPP_D (Karnataka)   

                         destination_name  start_scan_to_end_scan  \
0      Kanpur_Central_H_6 (Uttar Pradesh)                  2259.0   
1       Doddablpur_ChikaDPP_D (Karnataka)                   180.0   
2           Gurgaon_Bilaspur_HB (Haryana)                  3933.0   
3          Mumbai_MiraRd_IP (Maharashtra)                   100.0   
4           Sandur_WrdN1DPP_D (Karnataka)                   717.0   
...                                   ...                     ...   
14812      Chandigarh_Mehmdpur_H (Punjab)                   257.0   
14813      Faridabad_Blbgarh_DC (Haryana)                    60.0   
14814  Kanpur_GovndNgr_DC (Uttar Pradesh)                   421.0   
14815  Tirchchndr_Shnmgprm_D (Tamil Nadu)                   347.0   
14816       Sandur_WrdN1DPP_D (Karnataka)                   353.0   

       actual_distance_to_destination  actual_time  osrm_time  osrm_distance  \
0                          824.732854       1562.0      717.0       991.3523   
1                           73.186911        143.0       68.0        85.1110   
2                         1927.404273       3347.0     1740.0      2354.0665   
3                           17.175274         59.0       15.0        19.6800   
4                          127.448500        341.0      117.0       146.7918   
...                               ...          ...        ...            ...   
14812                       57.762332         83.0       62.0        73.4630   
14813                       15.513784         21.0       12.0        16.0882   
14814                       38.684839        282.0       48.0        58.9037   
14815                      134.723836        264.0      179.0       171.1103   
14816                       66.081533        275.0       68.0        80.5787   

       segment_actual_time  segment_osrm_time  segment_osrm_distance  
0                   1548.0             1008.0              1320.4733  
1                    141.0               65.0 

In [62]:
df2["trip_uuid"].value_counts().index.nunique()


14817